# Description

**TODO**

# Modules

In [1]:
library(reticulate)
pd <- import("pandas")

# Settings

In [2]:
# reference panel
REFERENCE_PANEL <- "GTEX_V8"
# REFERENCE_PANEL = "1000G"

# prediction models
## mashr
EQTL_MODEL <- "MASHR"

# ## elastic net
# EQTL_MODEL = "ELASTIC_NET"

chromosome <- NULL

In [3]:
# Parameters
chromosome = 18
REFERENCE_PANEL = "GTEX_V8"
EQTL_MODEL = "MASHR"


In [4]:
paste0("Using reference panel: ", REFERENCE_PANEL)

[1] "Using reference panel: GTEX_V8"

In [5]:
paste0("Using eQTL model: ", EQTL_MODEL)

[1] "Using eQTL model: MASHR"

In [6]:
# chromosome must be provided as parameter
stopifnot(!is.null(chromosome))

# Paths

In [7]:
GENE_CORRS_DIR <- Sys.getenv("PHENOPLIER_PHENOMEXCAN_LD_BLOCKS_GENE_CORRS_DIR")
IRdisplay::display(GENE_CORRS_DIR)

[1] "/opt/data/data/phenomexcan/ld_blocks/gene_corrs"

In [8]:
INPUT_DIR <- file.path(GENE_CORRS_DIR, tolower(REFERENCE_PANEL), tolower(EQTL_MODEL), "by_chr")
IRdisplay::display(INPUT_DIR)

[1] "/opt/data/data/phenomexcan/ld_blocks/gene_corrs/gtex_v8/mashr/by_chr"

In [9]:
INPUT_FILE <- file.path(INPUT_DIR, paste0("gene_corrs-chr", chromosome, ".pkl"))
IRdisplay::display(INPUT_FILE)
stopifnot(file.exists(INPUT_FILE))

[1] "/opt/data/data/phenomexcan/ld_blocks/gene_corrs/gtex_v8/mashr/by_chr/gene_corrs-chr18.pkl"

In [10]:
OUTPUT_DIR <- file.path(INPUT_DIR, "corrected_positive_definite")
IRdisplay::display(OUTPUT_DIR)
dir.create(OUTPUT_DIR, recursive = TRUE, showWarnings = FALSE)

[1] "/opt/data/data/phenomexcan/ld_blocks/gene_corrs/gtex_v8/mashr/by_chr/corrected_positive_definite"

In [11]:
OUTPUT_FILE <- file.path(OUTPUT_DIR, paste0("gene_corrs-chr", chromosome, ".pkl"))
IRdisplay::display(OUTPUT_FILE)
if (file.exists(OUTPUT_FILE)) {
  IRdisplay::display("Output file exists, it will be overwritten")
}

[1] "/opt/data/data/phenomexcan/ld_blocks/gene_corrs/gtex_v8/mashr/by_chr/corrected_positive_definite/gene_corrs-chr18.pkl"

[1] "Output file exists, it will be overwritten"

# Functions

In [12]:
# taken from https://www.r-bloggers.com/2013/08/correcting-a-pseudo-correlation-matrix-to-be-positive-semidefinite/
# TODO: add documentation
CorrectCM <- function(CM, p = 0) {
  n <- dim(var(CM))[1L]
  E <- eigen(CM)
  CM1 <- E$vectors %*% tcrossprod(diag(pmax(E$values, p), n), E$vectors)
  Balance <- diag(1 / sqrt(diag(CM1)))
  CM2 <- Balance %*% CM1 %*% Balance
  return(CM2)
}

In [13]:
is_positive_definite <- function(data) {
  eigenvalues <- eigen(data)$values
  nonpositive_eigenvalues <- eigenvalues[eigenvalues <= 0]

  if (length(nonpositive_eigenvalues) > 0) {
    IRdisplay::display("We need to correct the data and make the matrix positive definite")
    return(FALSE)
  } else {
    IRdisplay::display("Matrix is already positive definite!")
    return(TRUE)
  }
}

# Load data

In [14]:
gene_corrs <- pd$read_pickle(INPUT_FILE)

In [15]:
dim(gene_corrs)

[1] 96 96

In [16]:
head(gene_corrs[1:10, 1:10])

,ENSG00000079134,ENSG00000158270,ENSG00000176890,ENSG00000176105,ENSG00000080986,ENSG00000132205,ENSG00000101577,ENSG00000101608,ENSG00000118680,ENSG00000177426
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
ENSG00000079134,1.000000000,-0.001966216,-0.05780064,0.09583688,0.00251513,0.01702875,-0.089916668,0.06911504,-0.10781703,-0.007772298
ENSG00000158270,-0.001966216,1.000000000,-0.03519007,0.00557507,-0.02389234,-0.03518304,-0.005395476,0.01616782,-0.07419352,0.056954716
ENSG00000176890,-0.057800643,-0.035190068,1.00000000,0.17494935,0.04053366,0.07663973,-0.022326855,-0.03297596,-0.04554801,-0.075751358
ENSG00000176105,0.095836877,0.005575070,0.17494935,1.00000000,0.05667946,-0.02410930,0.028034362,0.15112287,0.03506153,0.103447248
ENSG00000080986,0.002515130,-0.023892336,0.04053366,0.05667946,1.00000000,-0.06066244,0.034296874,-0.04076309,0.05234749,-0.068280422
ENSG00000132205,0.017028747,-0.035183037,0.07663973,-0.02410930,-0.06066244,1.00000000,0.043703231,0.06657206,-0.05369115,0.057888750


# Check positive definiteness

In [17]:
is_positive_definite(gene_corrs)

[1] "We need to correct the data and make the matrix positive definite"

[1] FALSE

In [18]:
# see eigenvalues
eigenvalues <- eigen(gene_corrs)$values

In [19]:
nonpositive_eigenvalues <- eigenvalues[eigenvalues <= 0]
IRdisplay::display(length(nonpositive_eigenvalues))
IRdisplay::display(nonpositive_eigenvalues)

[1] 1

[1] -0.5137741

In [20]:
if (length(eigenvalues[eigenvalues <= 0]) == 0) { quit() }

# Make matrix positive definite if needed

In [21]:
gene_corrs_corrected <- CorrectCM(gene_corrs, 1)

In [22]:
dimnames(gene_corrs_corrected)[[1]] <- rownames(gene_corrs)

In [23]:
dimnames(gene_corrs_corrected)[[2]] <- colnames(gene_corrs)

In [24]:
gene_corrs_corrected <- as.data.frame(gene_corrs_corrected)

In [25]:
dim(gene_corrs_corrected)

[1] 96 96

# Check positive definiteness of corrected matrix

In [26]:
is_positive_definite(gene_corrs_corrected)

[1] "Matrix is already positive definite!"

[1] TRUE

In [27]:
# see eigenvalues
eigenvalues <- eigen(gene_corrs_corrected)$values

In [28]:
nonpositive_eigenvalues <- eigenvalues[eigenvalues <= 0]
IRdisplay::display(length(nonpositive_eigenvalues))
IRdisplay::display(nonpositive_eigenvalues)

[1] 0

numeric(0)

In [29]:
stopifnot(length(eigenvalues[eigenvalues <= 0]) == 0)

In [30]:
# quick and visual comparison of the two matrices
IRdisplay::display(head(gene_corrs[1:10, 1:10]))
IRdisplay::display(head(gene_corrs_corrected[1:10, 1:10]))

,ENSG00000079134,ENSG00000158270,ENSG00000176890,ENSG00000176105,ENSG00000080986,ENSG00000132205,ENSG00000101577,ENSG00000101608,ENSG00000118680,ENSG00000177426
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
ENSG00000079134,1.000000000,-0.001966216,-0.05780064,0.09583688,0.00251513,0.01702875,-0.089916668,0.06911504,-0.10781703,-0.007772298
ENSG00000158270,-0.001966216,1.000000000,-0.03519007,0.00557507,-0.02389234,-0.03518304,-0.005395476,0.01616782,-0.07419352,0.056954716
ENSG00000176890,-0.057800643,-0.035190068,1.00000000,0.17494935,0.04053366,0.07663973,-0.022326855,-0.03297596,-0.04554801,-0.075751358
ENSG00000176105,0.095836877,0.005575070,0.17494935,1.00000000,0.05667946,-0.02410930,0.028034362,0.15112287,0.03506153,0.103447248
ENSG00000080986,0.002515130,-0.023892336,0.04053366,0.05667946,1.00000000,-0.06066244,0.034296874,-0.04076309,0.05234749,-0.068280422
ENSG00000132205,0.017028747,-0.035183037,0.07663973,-0.02410930,-0.06066244,1.00000000,0.043703231,0.06657206,-0.05369115,0.057888750


,ENSG00000079134,ENSG00000158270,ENSG00000176890,ENSG00000176105,ENSG00000080986,ENSG00000132205,ENSG00000101577,ENSG00000101608,ENSG00000118680,ENSG00000177426
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
ENSG00000079134,1.000000000,0.019921166,0.002044198,0.034818418,-0.007736931,0.002405562,-0.053949950,0.07133247,-0.076877189,0.006152469
ENSG00000158270,0.019921166,1.000000000,-0.032053568,-0.002466983,-0.017982980,-0.006265369,-0.018098512,0.02964970,-0.027749594,0.021922666
ENSG00000176890,0.002044198,-0.032053568,1.000000000,0.066465937,0.033722487,0.022039829,0.002635233,0.01462445,-0.007495534,-0.017117671
ENSG00000176105,0.034818418,-0.002466983,0.066465937,1.000000000,0.026554965,0.016105815,0.001061563,0.06343425,-0.009814547,0.047078130
ENSG00000080986,-0.007736931,-0.017982980,0.033722487,0.026554965,1.000000000,-0.042706197,0.020050205,-0.03952233,0.025227840,-0.033197751
ENSG00000132205,0.002405562,-0.006265369,0.022039829,0.016105815,-0.042706197,1.000000000,0.005561773,0.04821691,-0.025189470,0.011938973


Both matrices should "look" similar. We are not interested in perfectly accurate correlation values (they are already inaccurate).

# Save

In [31]:
py_save_object(gene_corrs_corrected, OUTPUT_FILE)